In [32]:
# get max test id so far
import os
test_ids = [int(f.split("_")[1].split(".")[0]) for f in os.listdir("parsed_tests") if f.startswith("test_")]

max_test_id = max(test_ids)
max_test_id

967747

In [33]:
# Krok 1: Instalace potřebných knihoven
# Odkomentujte a spusťte následující řádek, pokud knihovny ještě nemáte nainstalované
# !pip install requests pdfplumber

import requests
import io
import pdfplumber
import re
import json
import os
from pprint import pprint

def analyzuj_test_z_pdf(url: str):
    """
    Stáhne PDF test z dané URL, vyparsuje z něj otázky a odpovědi
    a vrátí strukturovaná data.
    """
    print(f"Stahuji PDF z adresy: {url}...")

    try:
        # Stažení PDF souboru
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        print("PDF úspěšně staženo.")

        pdf_content = response.content

        # Extrakce textu z PDF
        print("Parsuji obsah PDF...")
        full_text = ""
        with io.BytesIO(pdf_content) as pdf_stream:
            with pdfplumber.open(pdf_stream) as pdf:
                for page in pdf.pages:
                    full_text += page.extract_text() + "\n"

        print("Extrakce textu dokončena. Zpracovávám otázky...")

        # Zpracování textu
        pattern_otazky = re.compile(r'(\d+)\.\s+([\s\S]+?)(?=Počet bodů: (\d))', re.MULTILINE)
        otazky = []
        matches = pattern_otazky.finditer(full_text)

        for match in matches:
            cislo_otazky = int(match.group(1))
            blok_otazky = match.group(2).strip()
            radky = blok_otazky.split('\n')
            text_otazky = radky[0]
            body_otazky = int(match.group(3))

            moznosti = {}
            spravna_odpoved = None

            # Regulární výraz pro odpovědi
            pattern_odpovedi = re.compile(r'^\s*([A-C])\.\s*(.*?)\s*$')

            for radek in radky[1:]:
                match_odpovedi = pattern_odpovedi.match(radek)
                if match_odpovedi:
                    pismeno = match_odpovedi.group(1)
                    zbytek_radku = match_odpovedi.group(2)

                    # Kontrola správné odpovědi (smajlík ☻)
                    if '☺' in zbytek_radku or '☻' in zbytek_radku:
                        spravna_odpoved = pismeno


                    # Vyčištění textu odpovědi od speciálních znaků
                    text_odpovedi = re.sub(r'[x☺☻●]', '', zbytek_radku).strip()
                    moznosti[pismeno] = text_odpovedi

            # Přidání pouze požadovaných dat
            otazky.append({
                "text_otazky": text_otazky,
                "moznosti": moznosti,
                "spravna_odpoved": spravna_odpoved,
                "body": body_otazky
            })

        print("Zpracování dokončeno.")
        return {
            "datum_testu": re.search(r'Datum ([\d.]+)', full_text).group(1),

            "prehled_otazek": otazky,
        }

    except requests.exceptions.RequestException as e:
        print(f"Chyba při stahování PDF: {e}")
        return None
    except Exception as e:
        print(f"Nastala neočekávaná chyba: {e}")
        return None



url_base = "https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id="

chyba_count = 0
for i in range(max_test_id, max_test_id+100):
    url_pdf_testu = url_base + str(i)

    # check if the if the test with this id exists in folder "parsed_tests"
    if f"test_{i}.json" in os.listdir("parsed_tests"):
        print("Skipuji zpracování, test již existuje.")
        continue
    print(f"\n\n--- Zpracovávám test s ID {i} ---")
    zpracovana_data = analyzuj_test_z_pdf(url_pdf_testu)
    if zpracovana_data is None:
        print(f"Test s ID {i} neexistuje nebo nastala chyba.")
        chyba_count += 1
        if chyba_count >= 5:
            print("Počet po sobě jdoucích chyb dosáhl 5, ukončuji zpracování.")
            break
        continue
    json.dump(zpracovana_data, open(f"parsed_tests/test_{i}.json", "w"), ensure_ascii=False, indent=4)



Skipuji zpracování, test již existuje.


--- Zpracovávám test s ID 967748 ---
Stahuji PDF z adresy: https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id=967748...
PDF úspěšně staženo.
Parsuji obsah PDF...
Extrakce textu dokončena. Zpracovávám otázky...
Zpracování dokončeno.


--- Zpracovávám test s ID 967749 ---
Stahuji PDF z adresy: https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id=967749...
PDF úspěšně staženo.
Parsuji obsah PDF...
Extrakce textu dokončena. Zpracovávám otázky...
Zpracování dokončeno.


--- Zpracovávám test s ID 967750 ---
Stahuji PDF z adresy: https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id=967750...
PDF úspěšně staženo.
Parsuji obsah PDF...
Extrakce textu dokončena. Zpracovávám otázky...
Zpracování dokončeno.


--- Zpracovávám test s ID 967751 ---
Stahuji PDF z adresy: https://zkouseni.laacr.cz/Zkouseni/PDFReport?module=M09&report=vysledek&id=967751...
PDF úspěšně staženo.
Parsuji obsah PDF...